# `s2Dcd` basic example
This is a quite simple example to illustrate how some functions of the module `s2Dcd` in practice.

You can find a Python source version of this code in the same folder of this notebook (or export is as you wish from a Jupyter notebook).




In [5]:
# Import "standard" modules
import os
import numpy as np
import geone as gn
import json
import copy

Import some modules contained in the `s2Dcd` package.

In [6]:
# Import non-standard modules
import s2Dcd.s2Dcd as s2Dcd
import s2Dcd.utili as utili

In [7]:
# Print header and start counting time
# (this is useful only for a standalone run, not in a Jupyter notebook...)
time_start = utili.print_start() 

# If you can use a parallel version of the MPS simulation core,
# define here the number of threads.
nthreads = 8



    ***********************
    ***  1059164934.py  ***
    ***********************
    START:  Tue Aug 16 17:35:07 2022


Then, the next step is to read all the information related to the training images (TIs) used. For the moment, we load the information as we would have when working with a complete 3D TI (that we don't have here, actually).
This information is contained in the `JSON` file `ti3Ddict.json`, which contains all the information needed to define the geometry of a 3D TI. See the module (geone)[https://github.com/randlab/geone] for more info about the image format.

In our case (see later in the script), we are using two (pseudo) 3D TIs:
- one along the plane $xz$, of dimensions $250{\times}1{\times}250$
- one along the plane $xy$, of dimensions $250{\times}250{\times}1$

Here no training image is provided for the plane $yz$, but if for your case study you have one, you can use it...

Therefore, ideally, the two aforementioned TIs could be two slices of a full 3D TI, of dimensions $250{\times}250{\times}250$. This is the information contained in the file `ti3Ddict.json`.


In [8]:
# %% Read some info about the TI

# Read from an external file the dimensions that would have a 3D TI
# (when composed by the 2D slices considered here)
ti3Ddict_file = "ti3Ddict.json"
with open(ti3Ddict_file, "r") as json_in:
    ti3Ddict = json.load(json_in)

# Use the parameters to create an empty 3D TI
ti3Ddict["val"] = np.full([ti3Ddict["nx"], ti3Ddict["ny"], ti3Ddict["nz"]],
                          np.nan)

With the lines above, the content of the `JSON` file is directly translated into a Python dictionary, that once properly initializated (for example with *NaN* values), can be directly used to create an `Image` object within `geone`:

In [9]:
ti3D = gn.img.Img(**ti3Ddict)

The next step is to read all the information related to the simulation grid and the simulation parameters.
This information is contained into the `JSON` file `ds3Ddict.json`.

In [10]:
# %% Read the simulation grid and the main simulation parameters    
#

# Create a dictionary containing the parameters that will be used to set up
# the default input parameters of the DeeSse
ds3Ddict_file = "ds3Ddict.json"
with open(ds3Ddict_file, "r") as json_in:
    ds3Din_dict = json.load(json_in)

Just to lighten the notation, use some variables...

In [11]:
nx = ds3Din_dict["nx"]    
ny = ds3Din_dict["ny"]
nz = ds3Din_dict["nz"]
sx = ds3Din_dict["sx"]
sy = ds3Din_dict["sy"]
sz = ds3Din_dict["sz"]
ox = ds3Din_dict["ox"]
oy = ds3Din_dict["oy"]
oz = ds3Din_dict["oz"]
nv = ds3Din_dict["nv"]
varname = ds3Din_dict["varname"]
name = "res3D.gslib"
# Add a 3D TI to the parameter file
ds3Din_dict["TI"] = np.array([ti3D])

From the information about the simulation grid retrieved above, we create an empty simulation grid initialized with *NaN*. This grid will be used as container for the results of the sequence of 2D simulation steps.

Here we do not provide conditioning data, but if you have some you can load in the proper location of this 3D container as they will be taken into account in the next steps of the alogoritm.

In [12]:
# %% Create an empty simulation grid to be filled with the sequential 2D simulations

# Create an empty simulation grid (Img) to be filled with the 2D simulations
val = np.full([ds3Din_dict["nx"], ds3Din_dict["ny"], ds3Din_dict["nz"]], np.nan)
res3D = gn.img.Img(nx, ny, nz, sx, sy, sz, ox, oy, oz, nv, val, varname, name)

# %% Set up the parameters and the training images

# Set up the general parameters to be used for the target 3D final result
ds3Din = gn.deesseinterface.DeesseInput(**ds3Din_dict)

# Set up the main parameters for the simulation along the plane *yz*
# from the 3D template (along this direction we do not have a TI in this example)
ds_yz_in = None

# Set up the main parameters for the simulation along the plane *xz*, starting
# from the 3D template
ds_xz_in = copy.deepcopy(ds3Din)
# For the moment we only set up the size of the SG. The origin will be defined
# within the simulation sequence.
ds_xz_in.ny = 1

# Read the training image along the plane xz
ti_xz = gn.img.readImageGslib(os.path.join("..","data","strebelle", 
                                           "ti_250x1x250.gslib"))
ds_xz_in.TI = np.array([ti_xz])

# Set up the main parameters for the simulation along the plane *xy*, starting
# from the 3D template
ds_xy_in = copy.deepcopy(ds3Din)
ds_xy_in.nz = 1
# Read the training image along the plane *xy*
ti_xy = gn.img.readImageGslib(os.path.join("..","data","strebelle", 
                                           "ti_250x250x1.gslib"))
ds_xy_in.TI = np.array([ti_xy])

# %% perform the simulation

# The max number of simulation steps to be performed. You can use it
# if you want to stop the simulation before the 3D domain is
# completed (i.e. for quick testing purposes...).
step_max = 1000000

# Print some simulation info
s2Dcd.print_sim_info(ds3Din, ds_yz_in, ds_xz_in, ds_xy_in, nthreads)

# Create the simulation sequence
seq = s2Dcd.create_seq(ds3Din, ds_yz_in, ds_xz_in, ds_xy_in, nthreads)

#
# Simulation
#
s2Dcd.sim_run(seq, step_max, res3D, ds3Din, nthreads)

# Stop counting time
utili.print_stop(time_start)


gn.img.writeImageVtk(res3D, "res3D.vtk", missing_value=-9999999)

# %% Print the result in 3D with PyVista
gn.imgplot3d.drawImage3D_surface(res3D, text='TI', scalar_bar_kwargs={'vertical':True})


    Dimension of the simulation domain: (52x71x64)
    Plane normal to axis *x*:
        No TI defined 
    Plane normal to axis *y*:
        MPS simulation engine: "DeeSse" with 8 threads
        TI file:               "../data/strebelle/ti_250x1x250.gslib"
    Plane normal to axis *z*:
        MPS simulation engine: "DeeSse" with 8 threads
        TI file:               "../data/strebelle/ti_250x250x1.gslib"

    *** Simulation starts *** 
    Seq.step   1 of 135 (max) - slice simulated: y =    0
* checking out license OK.
    Seq.step   2 of 135 (max) - slice simulated: z =    0
* checking out license OK.
    Seq.step   3 of 135 (max) - slice simulated: y =   70
* checking out license OK.
    Seq.step   4 of 135 (max) - slice simulated: z =   63
* checking out license OK.
    Seq.step   5 of 135 (max) - slice simulated: y =   35
* checking out license OK.
    Seq.step   6 of 135 (max) - slice simulated: z =   31
* checking out license OK.
    Seq.step   7 of 135 (max) - slice simul

* checking out license OK.
    Seq.step  93 of 135 (max) - slice simulated: y =   30
* checking out license OK.
    Seq.step  94 of 135 (max) - slice simulated: z =   28
* checking out license OK.
    Seq.step  95 of 135 (max) - slice simulated: y =   32
* checking out license OK.
    Seq.step  96 of 135 (max) - slice simulated: z =   30
* checking out license OK.
    Seq.step  97 of 135 (max) - slice simulated: y =   34
* checking out license OK.
    Seq.step  98 of 135 (max) - slice simulated: z =   32
* checking out license OK.
    Seq.step  99 of 135 (max) - slice simulated: y =   37
* checking out license OK.
    Seq.step 100 of 135 (max) - slice simulated: z =   34
* checking out license OK.
    Seq.step 101 of 135 (max) - slice simulated: y =   39
* checking out license OK.
    Seq.step 102 of 135 (max) - slice simulated: z =   36
* checking out license OK.
    Seq.step 103 of 135 (max) - slice simulated: y =   41
* checking out license OK.
    Seq.step 104 of 135 (max) - slice 

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)